# Week 4 — Part 03: Rate Limiting Lab

**Estimated time:** 60–90 minutes

---

## Pre-study (Self-learn)

Foundations Course assumes Self-learn is complete. If you need a refresher on production constraints:

- [Foundations Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Chapter 5: Resource Monitoring and Containerization](../self_learn/Chapters/5/Chapter5.md)

---

## What success looks like (end of Part 03)

- You can handle HTTP 429 (rate limit) errors gracefully.
- You can implement rate limiting in your client.
- You can back off when rate limits are hit.

### Checkpoint

After running this notebook:
- You can detect rate limit errors
- You can implement backoff strategies

## Learning Objectives

- Handle rate limit errors (HTTP 429)
- Implement rate limiting strategies
- Back off gracefully when limits are hit

### What this part covers
This notebook covers **rate limiting** — what happens when you send requests too fast, and how to handle it gracefully.

**HTTP 429 "Too Many Requests"** is the server's way of saying "slow down." Every LLM provider has rate limits:
- **RPM** (requests per minute) — how many calls you can make
- **TPM** (tokens per minute) — how many tokens you can send/receive
- **Concurrent** — how many simultaneous requests are allowed

**Two responses to a 429:**
1. **Wait and retry** — if the `Retry-After` header tells you how long to wait, respect it
2. **Degrade** — serve a cheaper/faster result rather than making the user wait

## Overview

Rate limits protect providers and enforce fair usage.

In this lab you will:

- interpret HTTP 429 and `Retry-After`
- implement a simple local token bucket limiter
- practice deterministic graceful degradation (shrink prompt / fallback model)

If you want the deeper rate-limiter theory, use the Self-learn links at the top of the notebook.

### What this cell does
Implements `parse_retry_after_seconds()` — parses the `Retry-After` HTTP header into a float number of seconds.

**Why parse this header?** When a server returns 429, it often includes `Retry-After: 5` meaning "wait 5 seconds before retrying." Respecting this is both polite and practical — it avoids wasting retries before the limit resets.

**What to notice:** The function handles the integer-seconds format (most common for LLM APIs). The HTTP spec also allows an HTTP-date format (`Wed, 21 Oct 2015 07:28:00 GMT`) — the test at the bottom shows this returns `None` since we don't support it here. In production, you'd want to handle both.

## HTTP 429

429 means “Too Many Requests”.

Client behavior:

- respect the `Retry-After` header if present
- otherwise backoff and retry

Graceful degradation options (choose based on your product):

- return a clear “busy, try later” message
- fall back to a cheaper/faster model
- reduce prompt size / requested output length
- serve a cached result if correctness allows

### What this cell does
Implements a **Token Bucket** rate limiter — a classic algorithm for client-side rate limiting.

**How it works:**
- The bucket holds up to `capacity` tokens
- Tokens refill at `refill_per_s` tokens per second
- Each request costs `cost` tokens (default 1.0)
- If the bucket has enough tokens → allow the request and deduct
- If not → deny the request (caller must wait or degrade)

**Why use a token bucket?** It allows short bursts (up to `capacity` requests at once) while enforcing a long-term average rate. This matches how most LLM provider limits work — you can burst briefly but must stay within the per-minute average.

**What to notice in the output:** The first 5 requests are allowed immediately (bucket starts full at capacity=5). Then requests are denied until tokens refill at 1/second. After 0.2s × 5 = 1s, one token has refilled.

In [ ]:
from __future__ import annotations

import time
from dataclasses import dataclass
from typing import Optional


def parse_retry_after_seconds(value: str) -> Optional[float]:
    """Parse Retry-After header.

    For Foundations Course, we support the most common form: integer seconds.
    (HTTP also allows an HTTP-date; production clients should support both.)
    """
    v = value.strip()
    if not v:
        return None
    try:
        seconds = int(v)
        return max(0.0, float(seconds))
    except ValueError:
        return None


print(parse_retry_after_seconds("2"))
print(parse_retry_after_seconds("0"))
print(parse_retry_after_seconds(""))
print(parse_retry_after_seconds("Wed, 21 Oct 2015 07:28:00 GMT"))  # unsupported here

### What this cell does
Implements `decide_on_429()` — the decision logic for what to do when you receive a 429 response.

**Decision tree:**
1. Does the response have a `Retry-After` header?
2. Is the wait time ≤ `max_wait_s`? → **wait** that long, then retry
3. Otherwise → **degrade** (return a cheaper/faster result now rather than making the user wait)

**What to notice in the output:**
- `Retry-After: 2` with `max_wait_s=5` → `("wait", 2.0)` — 2 seconds is acceptable
- `Retry-After: 60` with `max_wait_s=5` → `("degrade", None)` — 60 seconds is too long to wait
- No `Retry-After` → `("degrade", None)` — unknown wait time, degrade immediately

In [ ]:
@dataclass
class TokenBucket:
    capacity: float
    refill_per_s: float
    tokens: float
    last_refill_s: float

    @classmethod
    def create(cls, *, capacity: float, refill_per_s: float) -> "TokenBucket":
        now = time.time()
        return cls(capacity=capacity, refill_per_s=refill_per_s, tokens=capacity, last_refill_s=now)

    def _refill(self) -> None:
        now = time.time()
        dt = max(0.0, now - self.last_refill_s)
        self.tokens = min(self.capacity, self.tokens + dt * self.refill_per_s)
        self.last_refill_s = now

    def allow(self, cost: float = 1.0) -> bool:
        self._refill()
        if self.tokens >= cost:
            self.tokens -= cost
            return True
        return False


bucket = TokenBucket.create(capacity=5, refill_per_s=1.0)

for i in range(12):
    ok = bucket.allow(cost=1.0)
    print(f"i={i:02d} allowed={ok} tokens_left={bucket.tokens:.2f}")
    time.sleep(0.2)

## Graceful degradation (what to do when limited)

When you hit rate limits, you typically have two broad options:

- **Wait and retry** (respecting `Retry-After` if provided)
- **Degrade** (return something cheaper/faster or less precise)

Common degradation choices:

- Return a clear “busy, try later” message
- Fall back to a cheaper/faster model
- Reduce prompt size (trim history, remove low-value context)
- Reduce requested output length
- Serve a cached result (only if acceptable for correctness)

The correct choice depends on your product:

- For interactive UX, a fast partial answer may be better than waiting.
- For offline pipelines, retrying may be preferable to degrading quality.

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple


@dataclass(frozen=True)
class RateLimitResponse:
    status_code: int
    retry_after: Optional[str] = None


def decide_on_429(
    resp: RateLimitResponse,
    *,
    max_wait_s: float,
) -> Tuple[str, Optional[float]]:
    """Return (action, wait_seconds).

    action is one of: "wait", "degrade".
    """
    if resp.retry_after is not None:
        s = parse_retry_after_seconds(resp.retry_after)
        if s is not None and s <= max_wait_s:
            return ("wait", s)
    return ("degrade", None)


print(decide_on_429(RateLimitResponse(429, retry_after="2"), max_wait_s=5.0))
print(decide_on_429(RateLimitResponse(429, retry_after="60"), max_wait_s=5.0))
print(decide_on_429(RateLimitResponse(429, retry_after=None), max_wait_s=5.0))

In [ ]:
def degrade_request(prompt: str, *, max_chars: int) -> str:
    # TODO: implement a deterministic prompt shrinking strategy.
    # Requirements:
    # - Never exceed max_chars.
    # - Preserve the beginning of the prompt (often contains instructions).
    # - Consider preserving the end too (often contains the latest user input).
    if max_chars <= 0:
        return ""
    if len(prompt) <= max_chars:
        return prompt

    head = max(0, max_chars // 2)
    tail = max_chars - head
    if tail <= 0:
        return prompt[:max_chars]
    return prompt[:head] + prompt[-tail:]


def choose_fallback_model(primary: str) -> str:
    # TODO: implement a simple fallback mapping.
    # Example behavior:
    # - if primary is "gpt-4", fallback to "gpt-4-mini"
    # - otherwise fallback to a safe default
    mapping = {
        "gpt-4": "gpt-4-mini",
        "gpt-4o": "gpt-4o-mini",
    }
    return mapping.get(primary, "gpt-4o-mini")


print("degraded:", degrade_request("INSTRUCTIONS..." + ("x" * 200) + "...LATEST", max_chars=60))
print("fallback:", choose_fallback_model("gpt-4"))

## References

- HTTP 429: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

## Appendix: Solutions (peek only after trying)

Reference implementations for `degrade_request` and `choose_fallback_model`.

In [ ]:
def degrade_request(prompt: str, *, max_chars: int) -> str:
    if max_chars <= 0:
        return ""
    if len(prompt) <= max_chars:
        return prompt

    if max_chars <= 10:
        return prompt[:max_chars]

    keep_head = max(1, int(max_chars * 0.6))
    keep_tail = max_chars - keep_head
    return prompt[:keep_head] + prompt[-keep_tail:]


def choose_fallback_model(primary: str) -> str:
    mapping = {
        "gpt-4": "gpt-4-mini",
        "gpt-4o": "gpt-4o-mini",
        "claude-3-opus": "claude-3-sonnet",
    }
    return mapping.get(primary, "gpt-4o-mini")


print("degraded:", degrade_request("INSTRUCTIONS..." + ("x" * 200) + "...LATEST", max_chars=60))
print("fallback:", choose_fallback_model("gpt-4"))